# A computational example
This Jupyter Notebook accompanies the paper "Mukai lifting of self-dual points" by Barbara Betti and Leonie Kayser.
The code is in Julia language.
The folder $\texttt{MukaiLiftSource}$ contains all the files and we need to work inside this directory.
We describe how to use the package $\texttt{MukaiLiftP6}$.

In [1]:
using Pkg;
Pkg.activate("MukaiLiftP6")

  Activating project at `~/Documents/GitHub/Gr26/MukaiLiftSource/MukaiLiftP6`


In [2]:
using MukaiLiftP6
using Oscar
using LinearAlgebra
using HomotopyContinuation
using DelimitedFiles
using JLD


We define the configuration of points as a $7\times 14$ matrix $\Gamma$. 
We prove that it is self-dual by computing the matrix $\Lambda$ satisfying condition (ii) of Lemma 2.1 and checking that $\Gamma\cdot\Lambda\cdot\Gamma^\intercal=0$.

In [3]:
Gamma = [7 -2 6 -1 -6 1 -9 7 0 6 1 8 -3 7; -1 2 -5 -2 0 -4 3 -3 -4 -3 4 -2 4 -1; 1 4 -1 -5 -3 6 8 -1 -8 -3 5 1 -6 -8; 3 -6 4 -3 -4 6 0 5 8 2 3 2 -8 0; 1 -2 1 0 -4 2 2 3 4 -1 2 2 -2 -2; 0 -6 -5 6 3 7 -3 2 8 -7 -6 -3 -5 5; -3 3 -4 1 4 3 2 -3 -6 -4 -3 -4 -1 -2]
Lambda = certify_selfdual(Gamma)
norm( Gamma*Lambda*transpose(Gamma), Inf) 

2.6867397195928788e-14

We compute the Orthogonal normal form $\Gamma_{\text{ONF}}= [I_7 \mid P]$ of $\Gamma$ together with the linear projective transformation $A$ such that $A\cdot \Gamma_{\text{ONF}}= \Gamma\cdot \Lambda_{\text{scale}}$.

In [4]:
Gamma_ONF, A, Lambda_scale = orthogonal_normal_form(Gamma)
norm(A*Gamma_ONF - Gamma*Lambda_scale, Inf)

2.6645352591003757e-15

In [32]:
eigenvalues(Gamma_ONF[:,8:14])

7-element Vector{ComplexF64}:
  -0.9959219418402544 + 0.09021909864954644im
  -0.9959219418402541 - 0.0902190986495468im
  -0.7722875574482894 + 0.63527311340128im
  -0.7722875574482889 - 0.6352731134012801im
 -0.28251091380381904 - 0.9592640843801726im
 -0.28251091380381876 + 0.9592640843801731im
   0.9999999999999977 + 0.0im

The orthogonal normal form of $\Gamma$ is given by $[I_7 \mid P]$, where $P$ is the orthogonal matrix:

In [16]:
P = Matrix{Float64}(Gamma_ONF[:,8:14])

7×7 Matrix{Float64}:
 -0.339885   0.059965   -0.0924461  …   0.756392   -0.303323    0.391105
 -0.205214  -0.850587    0.0196552      0.26801     0.19422    -0.292163
  0.100775   0.230199    0.811776       0.314816   -0.13109    -0.320813
  0.094778   0.318981   -0.408428       0.329141    0.303156   -0.0306488
 -0.773382   0.0614641   0.155243      -0.374698   -0.160906    0.200883
  0.26268   -0.166903   -0.242692   …  -0.071958   -0.855751   -0.155243
 -0.395103   0.294162   -0.286833       0.0549679  -0.0546348  -0.770301

We compute the skew normal form $\Gamma_{\text{SNF}}$ of $\Gamma$ using the Cayley transform of $P$.

In [17]:
S = cayley(P)
Gamma_SNF = [I+S I-S]

7×14 Matrix{Float64}:
  1.0           -1.0   1.0  …    2.88903e-15   1.0   -4.0   -2.0
  1.0            1.0   3.0       2.0           2.0  -10.0  -12.0
 -1.0           -3.0   1.0      -2.0          -1.0    1.0    2.0
 -2.21248e-15    2.0  -2.0       1.0           1.0   10.0    6.0
  1.0            2.0  -1.0      -1.0           1.0    4.0    4.0
 -4.0          -10.0   1.0  …  -10.0          -4.0    1.0    6.0
 -2.0          -12.0   2.0      -6.0          -4.0   -6.0    1.0

The target parameters $S_{\text{target}}$ of the homotopy are the entries of the skew-symmetric matrix $S$.

In [18]:
S_target = skew_to_vector(S)

21-element Vector{Any}:
 -0.9999999999999908
  0.9999999999999988
 -2.889029324957673e-15
 -1.0000000000000036
  3.9999999999999964
  1.9999999999999964
  2.999999999999989
 -2.0000000000000133
 -2.0000000000000124
  9.999999999999957
  ⋮
  1.0000000000000038
 -0.9999999999999939
 -1.9999999999999991
 -0.9999999999999967
 -9.999999999999972
 -5.999999999999978
 -3.999999999999975
 -3.9999999999999787
 -6.0000000000000036

In [8]:
S_start, L_start = make_start();
parametrized_system, l_start, A_rand = make_poly_system(S_start, L_start);

We solve the mukai lifting problem for $\Gamma$.

In [9]:
@time result = HomotopyContinuation.solve(parametrized_system, l_start; compile = false, start_parameters=S_start, target_parameters=S_target)

4771.944745 seconds (209.56 M allocations: 41.882 GiB, 0.06% gc time, 0.36% compilation time)


Result with 1 solution
• 1 path tracked
• 1 non-singular solution (0 real)
• random_seed: 0x803386cf


We analyze the solution. There is one path tracked. The residual is the infinity norm of the vector with the equations evaluated in the computed solution and it is numerically zero.

In [10]:
sol = solutions(result)[1]
result.path_results

1-element Vector{PathResult}:
 PathResult:
 • return_code → :success
 • solution → ComplexF64[-0.273635458887796 - 0.06775243423895316im, -0.011846851652547076 + 0.09792162245375761im, 0.08663316597774245 + 0.24058737357966947im, 0.20161714542068646 - 0.30866469081893283im, -0.2915910352254546 - 0.3030373385426371im, 0.3111662900649619 + 0.06797542086680466im, -0.07827667453148025 - 0.1550983855487094im, 0.21062486225557217 + 0.059882506074793666im, 0.27856785167870723 + 0.35176144250022345im, 0.03276616552411386 - 0.12949461461760242im  …  -0.13268153478220437 + 0.17843363857781436im, 0.11049193985152128 + 0.387953715218807im, 0.23929147066151465 + 0.36600017655778im, 0.3030355026666242 + 0.04596677974744089im, -0.24840468511925562 + 0.013800304821779004im, 0.07017367334342992 + 0.0517129396163441im, -0.48551202054006604 + 0.11561748471892667im, -0.17421013064254257 - 0.1437684870112188im, -0.0010102994146811172 + 0.0886192569254505im, -0.14678405373738854 + 0.1590845099543955im]
 • a

The embedding $\widehat L\colon {\mathbb P}^6 \simeq \mathbb{L} \subseteq {\mathbb P}^{14}$ such that $\widehat L(\Gamma_{\text{SNF}})=\mathbb{L}\cap X_8$ is:

In [11]:
L_hat = L_start + sum(sol[i]*A_rand[i] for i=1:69)

15×7 Matrix{ComplexF64}:
  0.110358+1.48495im     -0.309519-0.312877im   …       0.325-0.752964im
   2.37038-0.999861im    -0.655104+0.301715im       -0.021756+0.146643im
 -0.366777-1.89787im     -0.340293+0.459462im       -0.165027-0.634101im
   -1.7371-1.35588im      0.568308-0.66831im          1.07132-0.907229im
 -0.859459-0.151393im     0.418066+0.268218im      -0.0948581-0.15461im
 -0.258905+2.2502im        1.11948-0.891757im   …     1.66824-0.930953im
   1.34186+0.681609im    -0.123419-0.381982im       -0.416171-0.636901im
  0.190423-1.69953im       -1.6864+0.795628im        -1.29985+1.10727im
 -0.433555-0.0729704im   -0.926099-0.567373im       -0.729298-0.20836im
 -0.343618-0.704737im   -0.0920986+0.366301im       -0.645506+0.309184im
  0.790917+1.62269im     -0.440553-1.02023im    …   -0.518834-1.88644im
  0.538597+0.889796im    -0.203765+0.192049im        0.177876+0.80178im
   1.56667-0.580213im     -0.55025+0.135477im       -0.931665+0.565031im
  0.581298-0.807628im    -0.457

Composing with the previous coordinate transformations $\Gamma \leadsto \Gamma_{\text{SNF}}$ we obtain $L \colon \mathbb{P}^6 \to \mathbb{P}^{14}$ such that $L(\Gamma)= \mathbb{L}\cap X_8$

In [12]:
L = L_hat*(I+S)*inv(A)

15×7 Matrix{ComplexF64}:
   -1.73316-1.13661im      -2.59528-2.78603im   …  0.277849-5.97082im
   -1.47082+0.123943im     -1.54641+1.39818im      -3.48991+0.241175im
 -0.0551945-0.282491im     0.170444-1.07227im        1.4313-2.14348im
    4.22987-0.0335093im     6.41355-0.722674im      12.4421+4.90968im
 -0.0762985+1.25169im     -0.636843+0.848057im     -1.61724+0.86904im
     3.0529-0.735421im      3.27292-3.69021im   …    8.8254+5.74991im
   -1.10684-0.615986im     -2.26215+0.750087im       2.8094-1.06477im
    -3.5943-2.76883im      -4.95795-1.67682im      -6.59014-6.82233im
   -1.15533-2.33476im      -1.02171-2.37178im       2.55248-9.21312im
   -0.93376+0.297742im    -0.709759+1.14198im      -1.87763-1.83187im
   0.395161-3.56589im      -2.30956-5.234im     …   8.20974-7.24393im
  -0.476015+0.4588im     -0.0406401+1.16992im       -4.6234+1.04385im
   -2.17368+0.388263im     -1.82299+2.5939im       -1.92343-3.11303im
  0.0280782-0.818669im    0.0289104-1.17448im       1.82625-0.59

We can verify $L(\Gamma) = \mathbb{L} \cap X_8$ by evaluating $L(\gamma_i)$ in the Plücker relations.

In [13]:
plück_oscar = gens( grassmann_pluecker_ideal(2,6))
q = poly_to_func(plück_oscar)
maximum([ norm(q(L*Gamma[:,i]) ,Inf) for i =1:14])

7.907992711624597e-12